In [2]:
import os
import re

with open("tol_models.lsp", "w") as f:
    f.write("(defun allinstance () \n")
    
    for folder in os.listdir("TOLdataset"):
        f.write("(reset) \n (initialize-model) \n")
        problem_file = "TOLdataset/" + folder + "/problem.pddl"
        output_file = "TOLdataset/" + folder + "/model_time.txt"
        
        problem = open(problem_file).read()
        init_string = problem[problem.find("(:init") + 6 :problem.find("(:goal")].strip() 
        infos = init_string.split("\n")[:-1]
        pos = [['nil'] * 3 for i in range(3)]
        for info in infos:
            if info.strip()[1] == 'a':
                tmp = info.strip()[1:-2].split('-')
                box = tmp[1]
                loc = tmp[2]
                
                x,y = loc.split('_')[1:]
                
                pos[int(x)-int(y)][2-int(x)] = box.split('_')[1]
        
        f.write('(spew t (puzzle :contents #2A(')
        
        for line in pos:
            f.write('(' + line[0] + ' ' + line[1] + ' ' + line[2] + ')\n')
            
        f.write('))\n')
        f.write('*weight*)\n')
        
        
        goal_string = problem[problem.find("(:goal"):-1].strip()
        infos = re.findall("at-b_\d-loc_\d_\d",goal_string)
        pos = [['nil'] * 3 for i in range(3)]
        for info in infos:
            if info.strip()[0] == 'a':
                tmp = info.strip().split('-')
                box = tmp[1]
                loc = tmp[2]
                
                x,y = loc.split('_')[1:]
                
                pos[int(x)-int(y)][2-int(x)] = box.split('_')[1]
        
        f.write('(spew t (end-puzzle :contents #2A(')
        
        for line in pos:
            f.write('(' + line[0] + ' ' + line[1] + ' ' + line[2] + ')\n')
            
        f.write('))\n')
        f.write('*weight*)\n')
        
        f.write('(run 1000)\n')
        f.write('(dm)\n')
        f.write('(with-open-file (stream "' + output_file + '" :direction :output)\n')
        f.write('(format stream "~A" *cycles*))\n')
        
        
    f.write(")")

In [7]:
import json, os, re

problem_ids = []
init_poss = {}
goal_poss = {}
anss = {}
second_num = {(0,0,3):'1',(1,0,2):'2',(0,1,2):'3',(0,2,1):'4',(1,1,1):'5',(1,2,0):'6'}
R = 'r'
G = 'g'
B = 'b'
ID2color = {11:{(2,0):R,(2,1):G, (2,2):B},12:{(2,0):R,(2,1):G,(0,0):B}, 13:{(2,0):R,(2,1):G,(1,0):B}, 14:{(2,0):R,(1,0):B,(1,1):G}, 15:{(2,0):R,(1,0):B,(0,0):G}, 16:{(1,0):B,(1,1):R,(0,0):G},
           21:{(2,0):R,(2,1):B, (2,2):G},22:{(2,0):R,(2,1):B,(0,0):G}, 23:{(2,0):R,(2,1):B,(1,0):G}, 24:{(2,0):R,(1,0):G,(1,1):B}, 25:{(2,0):R,(1,0):G,(0,0):B}, 26:{(1,0):G,(1,1):R,(0,0):B},
            31:{(2,0):B,(2,1):R, (2,2):G},32:{(2,0):B,(2,1):R,(0,0):G}, 33:{(2,0):B,(2,1):R,(1,0):G}, 34:{(2,0):B,(1,0):G,(1,1):R}, 35:{(2,0):B,(1,0):G,(0,0):R}, 36:{(1,0):G,(1,1):B,(0,0):R},
            41:{(2,0):B,(2,1):G, (2,2):R},42:{(2,0):B,(2,1):G,(0,0):R}, 43:{(2,0):B,(2,1):G,(1,0):R}, 44:{(2,0):B,(1,0):R,(1,1):G}, 45:{(2,0):B,(1,0):R,(0,0):G}, 46:{(1,0):R,(1,1):B,(0,0):G},
            51:{(2,0):G,(2,1):B, (2,2):R},52:{(2,0):G,(2,1):B,(0,0):R}, 53:{(2,0):G,(2,1):B,(1,0):R}, 54:{(2,0):G,(1,0):R,(1,1):B}, 55:{(2,0):G,(1,0):R,(0,0):B}, 56:{(1,0):R,(1,1):G,(0,0):B},
            61:{(2,0):G,(2,1):R, (2,2):B},62:{(2,0):G,(2,1):R,(0,0):B}, 63:{(2,0):G,(2,1):R,(1,0):B}, 64:{(2,0):G,(1,0):B,(1,1):R}, 65:{(2,0):G,(1,0):B,(0,0):R}, 66:{(1,0):B,(1,1):G,(0,0):R}}

data_string = open("test.txt").read().strip().split("\n\n")
data_items = [line.strip().split("\n") for line in data_string]

count = 0
for folder in os.listdir("TOLdataset"):
    problem_file = "TOLdataset/" + folder + "/problem.pddl"

    problem = open(problem_file).read()
    init_string = problem[problem.find("(:init") + 6 :problem.find("(:goal")].strip() 
    infos = init_string.split("\n")[:-1]
    pos = [[None] * 3 for i in range(3)]
    height_tuple = [0]*3
    for info in infos:
        if info.strip()[1] == 'a':
            tmp = info.strip()[1:-2].split('-')
            box = tmp[1]
            loc = tmp[2]

            x,y = loc.split('_')[1:]

            pos[int(x)][int(y)] = box.split('_')[1]

    if pos[0][0]:
        height_tuple[0] = 1
    if pos[1][1]:
        height_tuple[1] = 2
    elif pos[1][0]:
        height_tuple[1] = 1
        
    init_pos = [[0,0,0],[0,0],[0]]
    for i in range(len(init_pos)):
        for j in range(0,3-i):
            if pos[2-i][j]:
                init_pos[i][j] = int(pos[2-i][j]) + 1
                
    print(init_pos)
        
        
    height_tuple[2] = 3 - height_tuple[0] - height_tuple[1]
        
    num_2 = second_num[(height_tuple[0], height_tuple[1],height_tuple[2])]
    start_num = int('2' + num_2)
    color = ID2color[start_num]
    num2color = {}
    for (x,y) in color:
        num2color[pos[x][y]] = color[(x,y)]
    


    goal_string = problem[problem.find("(:goal"):-1].strip()
    infos = re.findall("at-b_\d-loc_\d_\d",goal_string)
    pos = [[None] * 3 for i in range(3)]
    for info in infos:
        if info.strip()[0] == 'a':
            tmp = info.strip().split('-')
            box = tmp[1]
            loc = tmp[2]

            x,y = loc.split('_')[1:]

            pos[int(x)][int(y)] = box.split('_')[1]
    
    goal_pos = [[0,0,0],[0,0],[0]]
    for i in range(len(goal_pos)):
        for j in range(0,3-i):
            if pos[2-i][j]:
                goal_pos[i][j] = int(pos[2-i][j]) + 1
                
    print(goal_pos)
            
    for num in ID2color:
        tmp = ID2color[num]
        for (x,y) in tmp:
            if (not pos[x][y]) or (not num2color[pos[x][y]] == tmp[(x,y)]):
                break
        else:
            end_num = num
            break
            
    with open("TOLdataset/" + folder + "/info.json") as f:
        info_dict = json.load(f)



    
    
    
    for line in data_items:
        if line[0] == str(start_num) and line[1] == str(end_num):
            print(folder)
            print(line)
            print(info_dict['optimal_cost(gc)'])
            print(info_dict['num_sols'])
            info_dict['num_sols'] = int(line[3])
            info_dict['alter_move'] = float(line[4])
            assert int(line[5]) == info_dict['start_hierarchy'] + 1
            assert int(line[6]) == info_dict['goal_hierarchy'] + 1
            
            info_dict['score'] = float(line[-1])
            problem_ids.append(folder)
            init_poss[folder] = init_pos
            goal_poss[folder] = goal_pos
            anss[folder] = info_dict['optimal_cost(gc)']
            count += 1
            print(count)
            break
    
    info_dict['init_pos'] = init_pos
    info_dict['goal_pos'] = goal_pos
            
    with open("TOLdataset/" + folder + "/info.json", 'w') as f:
        json.dump(info_dict, f)
    
print(problem_ids)
print(init_poss)
print()
print(goal_poss)
print()
print(anss)
#     f.write('(spew t (end-puzzle :contents #2A(')

#     for line in pos:
#         f.write('(' + line[0] + ' ' + line[1] + ' ' + line[2] + ')\n')

[[3, 0, 0], [1, 2], [0]]
[[1, 0, 0], [2, 3], [0]]
TOL_87
['24', '64', '5', '1', '3.08', '2', '2', '0.899', '0.545', '0.400', '2.792']
5
1
1
[[3, 0, 0], [2, 0], [1]]
[[3, 2, 0], [1, 0], [0]]
[[2, 3, 0], [1, 0], [0]]
[[2, 0, 0], [3, 1], [0]]
[[1, 2, 3], [0, 0], [0]]
[[2, 3, 0], [0, 0], [1]]
TOL_18
['21', '42', '5', '1', '3.00', '3', '2', '-0.515', '-0.170', '-1.044', '-0.533']
5
1
2
[[0, 0, 0], [2, 3], [1]]
[[1, 2, 0], [3, 0], [0]]
TOL_190
['26', '43', '4', '1', '3.20', '2', '2', '-0.598', '-0.297', '-0.738', '-1.158']
4
1
3
[[1, 2, 3], [0, 0], [0]]
[[0, 0, 0], [2, 1], [3]]
[[2, 3, 0], [0, 0], [1]]
[[0, 0, 0], [3, 2], [1]]
[[3, 0, 0], [2, 0], [1]]
[[1, 2, 3], [0, 0], [0]]
TOL_144
['25', '41', '5', '2', '3.00', '1', '3', '-0.667', '-0.680', '-0.374', '-0.588']
5
2
4
[[3, 0, 0], [2, 0], [1]]
[[2, 1, 0], [3, 0], [0]]
TOL_155
['25', '53', '7', '1', '3.38', '1', '2', '0.760', '0.888', '0.015', '1.007']
7
1
5
[[2, 3, 0], [0, 0], [1]]
[[3, 0, 0], [1, 2], [0]]
TOL_120
['22', '34', '4', '1', '3.2

In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
print(data_items)
scaler = StandardScaler()

X=np.array(data_items)

X_p = X[:,2:7].astype(float)

scaler.fit(X_p)

X_norm = scaler.transform(X_p)

scaler_y = StandardScaler() 
Y = X[:,-1].astype(float).reshape(-1,1)
scaler_y.fit(Y)
Y_norm = scaler_y.transform(Y)

model = LinearRegression()
model.fit(X_norm, Y_norm)
r1 = ['behavior']
r1.extend(model.coef_[0])

from prettytable import PrettyTable

t = PrettyTable(['Type', 'Minimum moves', 'Number of paths', 'Number of move choices', 'Start hierarchy', 'Goal hierarchy'], float_format = ".2")
t.add_row(r1)
print(t)

[['21', '11', '4', '2', '3.00', '3', '3', '-1.838', '-1.838', '-1.366', '-1.068'], ['21', '31', '6', '2', '3.00', '3', '3', '-0.284', '-0.249', '-0.228', '-0.300'], ['21', '32', '6', '2', '3.14', '3', '2', '0.737', '0.713', '1.182', '-0.819'], ['21', '33', '5', '2', '3.17', '3', '2', '-1.025', '-1.208', '0.186', '-1.756'], ['21', '34', '4', '1', '3.00', '3', '2', '-1.561', '-1.359', '-1.071', '-2.064'], ['21', '35', '4', '1', '3.00', '3', '1', '-1.139', '-0.994', '-0.886', '-1.271'], ['21', '41', '6', '1', '2.86', '3', '3', '0.525', '0.083', '1.375', '0.299'], ['21', '42', '5', '1', '3.00', '3', '2', '-0.515', '-0.170', '-1.044', '-0.533'], ['21', '43', '6', '1', '3.14', '3', '2', '0.301', '0.469', '-0.020', '-0.106'], ['21', '44', '7', '1', '3.13', '3', '2', '1.039', '1.357', '0.497', '-0.326'], ['21', '45', '7', '3', '3.25', '3', '1', '0.082', '0.094', '0.358', '-0.652'], ['21', '51', '7', '2', '2.88', '3', '3', '-0.045', '-0.224', '0.591', '-0.510'], ['21', '52', '6', '2', '3.00', '